# 11장. 심층 신경망 훈련하기

https://nbviewer.jupyter.org/github/rickiepark/handson-ml2/tree/master/

이 코드의 내용은 Hands-On Machine Learning with Scikit-Learn & TensorFlow을 참고했음을 밝힙니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 글로럿과 He 초기화

저자들은 적절한 신호가 흐르기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다고 주장합니다. 그리고 역방향에서 층을 통과하기 전과 후의 그레디언트 분산이 동일해야 합니다.

<img src='https://i.imgur.com/iBXgsbj.png' width='100%'>
(층의 입력과 출력 연결 개수를 fan-in과 fan-out이라고 부릅니다.)


<img src='https://i.imgur.com/57ifvaS.png' width='100%'>

In [ ]:
# kernel_initializer='he_uniform' or kernel_initializer='he_normal'

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [ ]:
# fan(in) 대신 fan(out) 기반의 균등분포 He 초기화

init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                          distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

# 수렴하지 않는 활성화 함수

### ReLU<br>
* 특정 양숫값에 수렴하지 않는다는 큰 장점
* 죽은 ReLU(dying ReLU): 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 의미에서 죽었다고 말합니다. 어떤 경우에는, 특히 큰 학습률을 사용하면 신경망의 뉴런 절반이 죽어 있기도 합니다. 뉴런의 가중치가 바뀌어 훈련 쎄트에 있는 모든 샘플에 대해 입력의 가중치 합이 음수가 되면 뉴런이 죽게 됩니다. 가중치 합이 음수이면 ReLU 함수의 그레디언트각 0이 되므로 경사 하강법이 더는 작동하지 않습니다.

### LeakyReLU
* LeakyReLU(a, z) = max(az, z)
* 하이퍼파라미터 a가 이 함수가 '새는(leaky)' 정도를 결정합니다. 새는 정도란 z<0일 때 이 함수의 기울기이며, 일반적으로 0.01로 설정합니다. 이 작은 기울기가 LeakyReLU를 절대 죽지 않게 만들어줍니다.
* 최근 한 논문에서 여러 ReLU 함수의 변종을 비교해 얻은 결론 하나는 LeakyReLU가 ReLU보다 항상 성능이 높다는 것입니다.
* 사실 a=0.2(많이 통과)로 하는 것이 a=0.01(조금 통과)보다 더 나은 성능을 내는 것으로 보입니다.


<img src='https://i.imgur.com/Ri6es6i.png' width='100%'>

### RReLU(randomized leaky ReLU)
* 훈련하는 동안 주어진 범위에서 a를 무작위로 선택하고 테스트시에는 평균을 사용

### PReLU(parametric leaky ReLU)
* a가 훈련하는 동안 하급
* 즉, 하이퍼파라미터가 아니고 다른 모델 파라미터와 마찬가지로 역전파에 의해 변경
* 대규모 이미지 데이텃셋에서는 ReLU보다 성능이 크게 앞섰지만, 소규모 데이턱셋에서는 훈려녀 세트에 과대적합될 위험 존재

### ELU(exponential linear unit)

*discussion*<br>
z < 0일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워진다?

* z < 0일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워집니다. 이는 앞서 이야기한 그레디언트 소실 문제를 완화해줍니다. 하이퍼파라미터 a는 z가 큰 음숫값일 때 ELU가 수렴할 값을 정의합니다. 보통 1로 설정하지만 다른 하이퍼파라미터처럼 변경할 수 있습니다.
* z < 0이어도 그레디언트가 0이 아니므로 죽은 뉴런을 만들지 않습니다.
* a = 1이면 이 함수는 z = 0에서 급격히 변동하지 않으므로 z = 0을 포함해 모든 구간에서 매끄러워 경사 하강법의 속도를 높여줍니다.
* ELU 활성화 함수의 주요 단점은 (지수 함수를 사용하므로) ReLU나 그 변종들보다 계산이 느리다는 것입니다. 훈련하는 동안에는 수렴 속도가 빨라서 느린 계산이 상쇄되지만 테스트 시에는 ELU를 사용한 네트워크가 ReLU를 사용한 네트워크보다 느릴 것입니다.

<img src='https://i.imgur.com/qAMjuxa.png' width='100%'>

<img src='https://i.imgur.com/qnO3YVa.png' width='100%'>

### SELU(Scaled ELU)
* 저자들은 완전 연결 층만 쌓아서 신경망을 만들고 모든 은닉층이 SELU 활성화 함수를 삿용한다면 네트워크가 자기 정규화(self-normalize)된다는 것을 보였습니다.
* 훈련하는 동안 각 층의 출력이 평균 0과 표준편차 1을 유지하는 경향이 있습니다. 이는 그레디언트 소싥과 폭주 문제를 막아줍니다.
* 그 결과로 SELU 활성화 함수는 이런 종류의 네트워크(특히 아주 깊은 네트워크)에서 다른 활성화 함수보다 뛰어난 성능을 종종 냅니다.
---
* 입력 특성이 반드시 표준화(평균 0, 표준편차 1)되어야 합니다.
* 모든 은닉층의 가중치는 르쿤 정규분포 초기화로 초기화되어야 합니다. 케라스에서는 kernel\_initializer='lecum\_normal'로 설정합니다.
* 네트워크는 일렬로 쌓은 층으로 구성되어야 합니다. 순환 신경망이나 스킵 연결(skip connection, 즉 와이드 & 딥 네트워크에서 건너뛰어 연결된 층)과 같은 순차적이지 않은 구조에 SELU를 사용하면 자기 정규화되는 것이 보장되지 않습니다. 따라서 SELU가 다른 활성화 함수보다 성능이 뛰어나지 않을 것입니다.

<img src='https://i.imgur.com/OASEpRa.png' width='100%'>

*tip*<br>
* 일반적: SELU > ELU > LeakyReLU(그리고 변종들) > ReLU > tanh > 로지스틱
* 네트워크가 자기 정규화되지 못하는 구조라면 ELU가 SELU보다 성능이 더 나을 수 있습니다.
* 실행 속도가 중요하다면 LeakyReLU를 선택할 수 있습니다. 하이퍼파라미터를 더 추가하고 싶지 않다면 케라스에서 사용하는 기본값 a를 사용합니다(예를 들어, LeakyReLU는 0.3).
* 시간과 컴퓨팅 파워가 충분하다면 교차 검증을 사용해 여러 활성화 함수를 평가해볼 수 있습니다.
* 신경망이 과대적합되었다면 RReLU
* 훈련 세트가 아주 크다면 PReLU
* ReLU가 (지금까지) 가장 널리 사용되는 활성화 함수이므로 많은 라이브러리와 하드웨어 가속기들은 ReLU에 특화되어 최적화되어 있습니다. 따라서 속도가 중요하다면 ReLU가 가장 좋은 선택일 것입니다.

In [ ]:
# he_normal
model = keras.models.Sequential([
    pass
    keras.layers.Dense(10, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2)
])

# selu
layer = keras.layer.Dense(10, activation='selu', kernel_initializer='lecun_normal')

# 배치 정규화

* 각 층에서 활성화 함수를 통과하기 전에나 후에 모데렝 연산을 하나 추가
* 이 연산은 단순하게 입력을 원점에 맞추고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터(하나는 스케일 조정에, 다른 하나는 이동에 사용)로 결괏값의 스케일을 조정하고 이동
* 많은 경우 신경망의 첫 번째 층으로 배치 정규화를 추가하면 훈련 세트를 (예를 들면 StandardScaler를 사용하여) 표준화할 필요가 없습니다. 배치 정규화 층이 이런 역할을 대신합니다.
* 그레디언트 소실 문제가 크게 감소하여 하이퍼볼릭 탄젠트나 로지스틱 활성화 함수 같은 수렴성을 가진 활성화 함수를 사용할 수 있습니다.
* 가중치 초기화에 네트워크가 훨씬 덜 민감해집니다.
* 저자들은 훨씬 큰 학습률을 사용하여 학습 과정의 속도를 크게 높일 수 있었습니다.
* 배치 정규화는 규제와 같은 역할을 하여 다른 규제 기법의 필요성을 줄여줍니다.
* 배치 정규화를 사용할 때 에포크마다 더 많은 시간이 걸리므로 훈련이 오히려 느려질 수 있습니다. 하지만 배치 정규화를 사용하면 수렴이 훨씬 빨라지므로 보통 상쇄됩니다. 따라서 더 적은 에포크로 동일한 성능에 도달할 수 있습니다.
---
* 모델의 복잡도를 키웁니다.
* 실행 시간 면에서도 손해입니다. 층마다 추가되는 계산이 신경망의 예측을 느리게 합니다.

<img src='https://i.imgur.com/fxF9DRt.png' width='100%'>

* $\gamma$(출력 스케일 벡터)와 $\beta$(출력 이동 벡터)는 일반적인 역전파를 통해 학습됩니다.
* $\mu$(최종 입력 평균 벡터)와 $\sigma$(최종 입력 표준편차 벡터)는 지수 이동 평균을 사용하여 추정됩니다.
* $\mu$와 $\sigma$는 훈련하는 동안 추정되지만 훈련이 끝난 후에 사용됩니다.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

* 배치 정규화 층은 입력마다 네 개의 파라미터 $\gamma$, $\beta$, $\mu$, $\sigma$를 추가합니다(예를 들면 첫 번째 배치 정규화 층은 4x784=3136개의 파라미터가 있습니다).
* 마지막 두 개의 파라미터 $\mu$와 $\sigma$는 이동 평균입니다. 이 파라미터는 역전파로 학습되지 않기 때문에 케라스는 'Non-trainable' 파라미터로 분류합니다(배치 정규화 파라미터의 전체 개수는 3136+1200+400입니다. 이를 2로 나누면 이 모델에서 훈련되지 않는 전체 파라미터 개수 2386을 얻습니다).
* 배치 정규화 논문의 저자들은 활성화 함수 이후보다 활성화 함수 이전에 배치 정규화 층을 추가하는 것이 좋다고 조언합니다. 하지만 작업에 따라 선호되는 방식이 달라서 이 조언에 대해서는 논란이 조금 있습니다.

In [ ]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

* 활성화 함수 전에 배치 정규화 층을 추가하려면 은닉층에서 활성화 함수를 지정하지 말고 배치 정규화 층 뒤에 별도의 층으로 추가해야 합니다.
* 배치 정규화 층은 입력마다 이동 파라미터를 포함하기 때문에 이전 층에서 편향을 뺄 수 있습니다(층을 만들 때 use_bias=False로 설정합니다).

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

BatchNormalization 클래스는 조정할 하이퍼파라미터가 적습니다. 보통 기본값이 잘 동작하지만 momentum 매개변수를 변경해야 할 수 있습니다. BatchNormalization 층이 지수 이동 평균을 업데이트할 때 이 하이퍼파라미터를 사용합니다. 새로운 값 v(즉, 현재 배치에서 계산한 새로운 입력 평균 벡터나 표준편차 벡터)가 주어지면 다음 식을 사용해 이동 평균 v_hat을 업데이트합니다. 적절한 모멘텀 값은 일반적으로 1에 가깝습니다. 예를 들면 0.9, 0.99, 0.999입니다(데이터셋이 크고 미니배치가 작으면 소수점 뒤에 9를 더 넣어 1에 가깝게 합니다).

<img src='https://i.imgur.com/OFPH9Du.png' width='100%'>

# 그레디언트 클리핑

그레디언트 폭주 문제를 완화하는 인기 있는 다른 방법은 역전파될 때 일정 임계값을 넘어서지 못하게 그레디언트를 잘라내는 것입니다. 이를 그레디언트 클리핑(gradient clipping)이라고 합니다. 순환 신경망은 배치 정규화를 적용하기 어려워서 이 방법을 많이 사용합니다. 다른 종류의 네트워크는 배치 정규화면 충분합니다.

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.complie(loss='mse', optimizer=optimizer)

이 옵티마이저는 그레디언트 벡터의 모든 원소를 -1.0과 1.0 사이로 클리핑합니다. 즉 (훈련되는 각 파라미터에 대한) 손실의 모든 편미분 값을 -1.0에서 1.0으로 잘라냅니다. 임계값은 하이퍼파라미터로 튜닝할 수 있습니다. 이 기능은 그레디언트 벡터의 방향을 바꿀 수 있습니다. 예를 들어 원래 그레디언트 벡터가 [0.9, 100.0]라면 대부분 두 번째 축 방향을 향합니다. 하지만 값을 기준으로 이를 클리핑을 하면 [0.9, 1.0]이 되고 거의 두 축 사이 대각선 방향을 향합니다. 실전에서는 이 방식이 잘 동작합니다. 만약 그레디언트 클리핑이 그레디언트 벡터의 방향을 바꾸지 못하게 하려면 clipvalue 대신 clipnorm을 지정하여 노름으로 클리핑해야 합니다. 만약 L2 노름이 지정한 임계값보다 크면 전체 그레디언트를 클리핑합니다. 예를 들어 clipnorm=1.0으로 지정한다면 벡터 [0.9, 100.0]이 [0.00899964, 0.9999595]로 클리핑되므로 방향을 그대로 유지합니다. 하지만 첫 번째 원소는 거의 무시됩니다. 훈련하는동안 그레디언트가 폭주한다면 다른 임계값으로 값과 노름을 모두 사용하여 클리핑할 수 있습니다(텐서보드를 사용해 그레디언트의 크기를 추적할 수 있습니다). 그리고 검증 세트에서 어떤 방식이 가장 좋은 성과를 내는지 확인할 수 있습니다.

*discussion*<br>
clipvalue를 사용하면 벡터의 방향이 바뀌는데 실전에서 잘 동작하는 이유?

# 사전훈련된 층 재사용하기

428~